In [ ]:
import boto3
from botocore.config import Config
import botocore.handlers
import re
import xarray as xr
import fsspec  # <-- this is important

aws_access_key_id = "LB2O0QEI4X544VA68HBP"
aws_secret_access_key = "QXQnrbIclgFv3aHura0GfRZALSQPigGer6fZPf9u"

#botocore.handlers.VALID_BUCKET = re.compile(r'^[:a-zA-Z0-9.\-_]{1,255}$')

# Setup
endpoint_url = 'https://eodata.dataspace.copernicus.eu'

# boto3 for listing
s3_client = boto3.client(
    's3',
    endpoint_url=endpoint_url,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

bucket = "eodata"
prefix = "Sentinel-5P/AUX/AUX_CTMANA/2018/09/"

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)

# Print object keys and pick one URL
for obj in response.get("Contents", []):
    key = obj["Key"]
    url = f"s3://{bucket}/{key}"  # <-- NOTE: fsspec needs s3:// URL, not HTTPS

# fsspec for opening the object
fs = fsspec.filesystem('s3', 
                       key=aws_access_key_id, 
                       secret=aws_secret_access_key,
                       client_kwargs={'endpoint_url': endpoint_url})

# Open file from S3 and load with xarray
with fs.open(url) as f:
    ds = xr.open_dataset(f, engine='h5netcdf')



In [2]:
ds

<xarray.Dataset> Size: 2GB
Dimensions:                                                (lon: 360, lat: 180,
                                                            levi: 35, lev: 34,
                                                            time: 48,
                                                            datelen: 6,
                                                            nGroundPixels: 450)
Coordinates:
  * lon                                                    (lon) float32 1kB ...
  * lat                                                    (lat) float32 720B ...
  * lev                                                    (lev) float32 136B ...
  * time                                                   (time) datetime64[ns] 384B ...
Dimensions without coordinates: levi, datelen, nGroundPixels
Data variables: (12/13)
    hyai                                                   (levi) float32 140B ...
    hybi                                                   (levi) float32 140B ...
    hyam                                                   (lev) float32 136B ...
    hybm                                                   (lev) float32 136B ...
    date                                                   (time, datelen) float32 1kB ...
    ps                                                     (time, lat, lon) float32 12MB ...
    ...                                                     ...
    surface_altitude                                       (lat, lon) float32 259kB ...
    nitrogendioxide_slant_column_density_stripe_amplitude  (nGroundPixels) float32 2kB ...
    tropopause_layer_index                                 (time, lat, lon) int32 12MB ...
    so2                                                    (time, lev, lat, lon) float32 423MB ...
    no2                                                    (time, lev, lat, lon) float32 423MB ...
    ch2o                                                   (time, lev, lat, lon) float32 423MB ...
Attributes: (12/26)
    Conventions:          CF-1.6
    validity_start:       20180930T000000
    validity_stop:        20181001T000000
    creation_date:        20221227T082925
    version:              mp 1 beta (benchmark)
    institution:          KNMI, Netherlands
    ...                   ...
    ybeg:                 -90
    yend:                 90
    zbeg:                 0
    zend:                 34
    meteo_model:          forecast (IFS)
    history:              Created 2022-12-27 08:29:25 +0000 by Henk Eskes wit...